In [316]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [317]:
# df = pd.read_csv('/content/drive/MyDrive/OpenBCISession_jakub_eeg1/OpenBCI-RAW-2021-05-15_13-30-02.txt', header=0, skiprows=4)
# df

In [318]:
df = pd.read_csv('/content/drive/MyDrive/drive-download-20210522T225648Z-001/mati_calculated1.csv', header=None, skiprows=1, names=[f'channel_{i}' for i in range(1,9)]+['trigger'])
df

,channel_1,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,channel_8,trigger
0,-5.358760,10.585452,-11.344654,27.494776,1.800974,-9.051285,-4.436563,-3.384233,0
1,-108.249093,1.170705,-129.186924,196.272772,-59.764140,-183.476812,-118.055441,-127.788794,0
2,-155.827190,-15.116721,-179.002972,256.830141,-94.212925,-262.576762,-171.932950,-188.503989,0
3,-134.736401,-32.388223,-147.280811,195.837731,-90.974491,-223.875483,-151.499973,-168.728158,0
4,-82.770224,-37.432544,-83.635659,100.216649,-63.876916,-132.551040,-96.691723,-109.642422,0
...,...,...,...,...,...,...,...,...,...
234197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,happiness
234198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,happiness
234199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,happiness
234200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,happiness


In [319]:
set(df['trigger'].to_list())

{'-1', '0', 'disgust', 'fear', 'happiness', 'sadness'}

In [320]:
df = df[(df['trigger'] == 'happiness') | (df['trigger'] == 'neutral') | (df['trigger'] == 'sadness')]
# df = df[(df['trigger'] == 'happiness') | (df['trigger'] == 'neutral')]

df = df.dropna().reset_index(drop=True)
df

,channel_1,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,channel_8,trigger
0,1.932648,3.451453,0.307611,5.205482,-1.948228,-1.530368,4.491480,-10.778688,happiness
1,4.848614,8.707081,3.827748,10.606331,1.148779,-2.302578,7.572053,-8.653732,happiness
2,4.892152,12.664270,3.699268,18.924472,2.229355,-6.112504,5.924430,-9.333093,happiness
3,4.489582,12.329430,3.450705,20.923349,1.634006,-6.593994,3.036260,-10.020900,happiness
4,5.244893,8.642658,4.910080,14.859411,1.351322,-0.323941,2.341524,-7.962567,happiness
...,...,...,...,...,...,...,...,...,...
139662,-48.352167,177.332190,-122.611844,408.251052,77.939955,-71.059523,27.089976,-64.063511,happiness
139663,-62.390964,246.672241,-168.043441,563.737408,108.880793,-95.836430,30.476504,-81.484548,happiness
139664,-67.125795,264.235109,-181.138369,603.912687,115.040376,-102.797171,29.951928,-86.220402,happiness
139665,-49.114061,181.729982,-127.269378,417.253644,76.848902,-72.783806,21.255610,-61.925171,happiness


In [321]:
def extract_features(x):
  t = 4

  AMP = x.max()
  LAT = x.idxmax()*t
  LAR = float(LAT/AMP)
  AAMP = abs(AMP)
  ALAR = math.fabs(LAR)
  PAR = x.where(x>0).sum()
  NAR = x.where(x<0).sum()
  TAR = PAR+NAR
  ATAR = abs(TAR)
  TAAR = PAR + abs(NAR)
  AASS = sum([(1/t)*math.fabs(x[i+1] - x[i]) for i in range(len(x)-1)])/len(x)
  PP = x.max() - x.min()
  PPT = abs(x.idxmax()*t - x.idxmin()*t)
  PPS = PP/PPT
  if x.idxmax() > x.idxmin():
    ZC = sum([1 for i in range(len(x[x.idxmin():x.idxmax()])-1) if (x[i] < 0 and x[i+1]>0) or (x[i]>0 and x[i+1]<0)])
  else:
    ZC = sum([1 for i in range(len(x[x.idxmax():x.idxmin()])-1) if (x[i] < 0 and x[i+1]>0) or (x[i]>0 and x[i+1]<0)])
  ZCD = ZC/PPT
  SSA = sum([0.5*math.fabs(
                          ((x[i-1] - x[i])/
                          math.fabs(x[i+1] - x[i]))
                          +
                          ((x[i-1] - x[i])/
                          math.fabs(x[i+1] - x[i]))) for i in range(1,len(x)-1)])

  after_fft = 2 * abs(np.fft.fft(x)) / len(x)
  f = np.linspace(0, 250, len(x))
  a = pd.DataFrame([after_fft, f]).T
  FFT_MODE = a[1][a[0][:int(len(x)/2)].idxmax()]
  FFT_MED = 0
  FFT_MEAN = sum([i * y for i,y in enumerate(after_fft)])/sum(after_fft)
  min_diff = 1000
  for i in range(1,len(after_fft)):
    diff = math.fabs(after_fft[:i].sum() - after_fft[i:].sum())
    if min_diff>diff:
      min_diff = diff
      FFT_MED = i
  return [AMP, LAT, LAR, AAMP, ALAR, PAR, NAR, TAR, ATAR, TAAR, AASS, PP, PPT, PPS, ZC, ZCD, SSA, FFT_MODE, FFT_MEAN, FFT_MED]

In [322]:
df.columns[:-1]

Index(['channel_1', 'channel_2', 'channel_3', 'channel_4', 'channel_5',
       'channel_6', 'channel_7', 'channel_8'],
      dtype='object')

In [323]:
all_lists = []
for col in df.columns[:-1]:
  row_list = []
  for second_end in range(0, len(df[col]-250), 250):
    row_list.append(extract_features(df[col][second_end:second_end+250].reset_index(drop=True)))
  all_lists.append(row_list)

In [324]:
feature_types = ['AMP', 'LAT', 'LAR', 'AAMP', 'ALAR', 'PAR', 'NAR', 'TAR', 'ATAR', 'TAAR', 'AASS', 'PP', 'PPT', 'PPS', 'ZC', 'ZCD', 'SSA', 'FFT_MODE', 'FFT_MEAN', 'FFT_MED']

colDataFrames = [pd.DataFrame(all_lists[i], columns=[f'{df.columns[i]}_{q}' for q in feature_types]) for i in range(1, len(all_lists))]

In [325]:
all_df = pd.concat(colDataFrames, axis=1)

In [326]:
all_df

,channel_2_AMP,channel_2_LAT,channel_2_LAR,channel_2_AAMP,channel_2_ALAR,channel_2_PAR,channel_2_NAR,channel_2_TAR,channel_2_ATAR,channel_2_TAAR,channel_2_AASS,channel_2_PP,channel_2_PPT,channel_2_PPS,channel_2_ZC,channel_2_ZCD,channel_2_SSA,channel_2_FFT_MODE,channel_2_FFT_MEAN,channel_2_FFT_MED,channel_3_AMP,channel_3_LAT,channel_3_LAR,channel_3_AAMP,channel_3_ALAR,channel_3_PAR,channel_3_NAR,channel_3_TAR,channel_3_ATAR,channel_3_TAAR,channel_3_AASS,channel_3_PP,channel_3_PPT,channel_3_PPS,channel_3_ZC,channel_3_ZCD,channel_3_SSA,channel_3_FFT_MODE,channel_3_FFT_MEAN,channel_3_FFT_MED,...,channel_7_AMP,channel_7_LAT,channel_7_LAR,channel_7_AAMP,channel_7_ALAR,channel_7_PAR,channel_7_NAR,channel_7_TAR,channel_7_ATAR,channel_7_TAAR,channel_7_AASS,channel_7_PP,channel_7_PPT,channel_7_PPS,channel_7_ZC,channel_7_ZCD,channel_7_SSA,channel_7_FFT_MODE,channel_7_FFT_MEAN,channel_7_FFT_MED,channel_8_AMP,channel_8_LAT,channel_8_LAR,channel_8_AAMP,channel_8_ALAR,channel_8_PAR,channel_8_NAR,channel_8_TAR,channel_8_ATAR,channel_8_TAAR,channel_8_AASS,channel_8_PP,channel_8_PPT,channel_8_PPS,channel_8_ZC,channel_8_ZCD,channel_8_SSA,channel_8_FFT_MODE,channel_8_FFT_MEAN,channel_8_FFT_MED
0,27.667216,432,15.614148,27.667216,15.614148,780.593418,-844.059832,-63.466414,63.466414,1624.653250,0.463464,43.810502,100,0.438105,1,0.010000,513.550903,7.028112,124.384959,104,18.259421,428,23.439955,18.259421,23.439955,728.434227,-685.872174,42.562053,42.562053,1414.306401,0.427309,33.349903,216,0.154398,5,0.023148,607.034849,7.028112,124.542107,103,...,19.494261,304,15.594333,19.494261,15.594333,722.370753,-755.406719,-33.035966,33.035966,1477.777472,0.666739,36.131432,92,0.392733,5,0.054348,1026.420573,7.028112,124.729144,112,16.233412,432,26.611781,16.233412,26.611781,667.080513,-767.095641,-100.015128,100.015128,1434.176153,0.459161,29.304804,60,0.488413,0,0.000000,602.182829,7.028112,124.074724,119
1,12.139609,384,31.631991,12.139609,31.631991,554.004247,-522.396227,31.608020,31.608020,1076.400474,0.413718,26.799601,56,0.478564,1,0.017857,1089.671354,7.028112,124.611596,114,14.595500,700,47.959987,14.595500,47.959987,534.484687,-540.185818,-5.701131,5.701131,1074.670506,0.352328,28.228892,372,0.075884,11,0.029570,519.319296,3.012048,124.929426,125,...,13.975997,700,50.085873,13.975997,50.085873,642.012171,-706.887489,-64.875318,64.875318,1348.899659,0.576051,33.762209,44,0.767323,2,0.045455,888.410636,5.020080,124.426602,103,11.087879,208,18.759223,11.087879,18.759223,560.624415,-595.976620,-35.352205,35.352205,1156.601035,0.390254,30.345744,124,0.244724,1,0.008065,529.394240,5.020080,124.599145,95
2,10.087100,176,17.448028,10.087100,17.448028,445.661860,-427.237963,18.423896,18.423896,872.899823,0.356177,21.456831,60,0.357614,1,0.016667,549.456642,6.024096,124.732381,103,11.444846,400,34.950229,11.444846,34.950229,465.813605,-420.109115,45.704491,45.704491,885.922720,0.319670,22.138300,84,0.263551,3,0.035714,732.898847,5.020080,124.377265,119,...,16.858268,136,8.067258,16.858268,8.067258,770.751569,-649.979945,120.771624,120.771624,1420.731514,0.610710,34.911226,124,0.281542,2,0.016129,540.029173,5.020080,123.979595,107,14.042276,8,0.569708,14.042276,0.569708,694.324107,-511.809851,182.514257,182.514257,1206.133958,0.372931,29.577642,428,0.069107,9,0.021028,3345.645741,5.020080,122.964775,67
3,21.281825,432,20.299011,21.281825,20.299011,1189.705191,-1119.723402,69.981789,69.981789,2309.428592,0.465130,63.724755,144,0.442533,7,0.048611,758.592225,3.012048,124.509228,117,20.407308,444,21.756911,20.407308,21.756911,1188.281333,-1199.065393,-10.784060,10.784060,2387.346726,0.372505,59.729557,128,0.466637,3,0.023438,621.170352,3.012048,124.915752,124,...,17.702457,384,21.691904,17.702457,21.691904,854.200061,-800.280622,53.919440,53.919440,1654.480683,0.524126,35.595998,400,0.088990,14,0.035000,627.353098,2.008032,124.520487,60,15.915426,880,55.292268,15.915426,55.292268,614.367466,-640.495292,-26.127826,26.127826,1254.862757,0.357919,34.107928,96,0.3

In [327]:
all_df['trigger'] = [df['trigger'][i] for i in range(0,len(df['trigger']), 250)]

In [328]:
all_df.to_csv('/content/drive/MyDrive/output1.csv', index=False)